In [126]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time
import datetime as dt 
import pymongo
from pymongo import MongoClient
import numpy as np

In [ ]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [ ]:
trendings_keywords = pd.read_csv('../Data/trendings.csv')

In [145]:
def get_trendings(trending, start_date, number_days, minutes_spam ):
    trending_words = '(' +trending+  ') -is:retweet'
    df_list = []
    end_date = start_date + relativedelta(days=number_days)
    start_time = dt.datetime.now()
    while start_date < end_date:
        now = dt.datetime.now()
        print('Extraction Date:', start_date, 'Trending:',trending, 'Start Time',start_time,  'Time extraction:',now, end="\r")
        upper_limit = start_date+ relativedelta(minutes=minutes_spam)
        start= start_date.isoformat("T") + "Z"
        end = upper_limit.isoformat("T") + "Z"
        rsp = client.search_all_tweets(query = trending_words ,start_time = start, end_time = end, sort_order= 'relevancy',
                                     tweet_fields = fields, max_results=500)
        time.sleep(1)
        tweets_df = pd.DataFrame(rsp.data, columns = fields)
        tweets_df['date']= start
        tweets_df = pd.concat([tweets_df.drop(['public_metrics'], axis=1), pd.json_normalize(tweets_df['public_metrics'])], axis=1)
        tweets_df = pd.concat([tweets_df.drop(['geo'], axis=1), pd.json_normalize(tweets_df['geo'])], axis=1)
        df_list.append(tweets_df)
        start_date = upper_limit
    trendings_df = pd.concat(df_list)
    trendings_df = trendings_df.reset_index(drop=True)
    return(trendings_df)

In [174]:
for i in range(len(trendings_keywords)):
    fields = ['lang',
     'public_metrics',
     'author_id',
     'id',
     'created_at',
     'text',
             'conversation_id',
             'geo',
             'in_reply_to_user_id']
    start_date = trendings_keywords['date'][0]
    newpath = '../Data/Trendings/'+ start_date + '/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    trending = trendings_keywords['trending'][0]
    file_name = start_date + '__' +  trending + '.gzip'
    if file_name not in os.listdir(newpath):
        number_days = 1
        minutes_spam = 10
        start_date_format = datetime.strptime(start_date, '%Y-%m-%d')
        trendings_df  = get_trendings(trending, start_date_format, number_days, minutes_spam )
        trendings_df.to_parquet(newpath + file_name,
                  compression='gzip')

In [176]:
pd.read_parquet(newpath + file_name, engine='pyarrow')

,lang,author_id,id,created_at,text,conversation_id,in_reply_to_user_id,date,retweet_count,reply_count,like_count,quote_count,place_id
0,es,1143717800,1528164731630018561,2022-05-22 00:03:58+00:00,No le creo al Registrador Vega. \n#NosUnimosON...,1528164731630018561,NaN,2022-05-22T00:00:00Z,27.0,1.0,41.0,0.0,None
1,es,774646729,1528165677760229376,2022-05-22 00:07:44+00:00,@Eganbernal @FicoGutierrez #NosUnimosONosJodem...,1528119885917831176,3.808014e+09,2022-05-22T00:00:00Z,24.0,0.0,42.0,0.0,None
2,es,93504770,1528163832690003969,2022-05-22 00:00:24+00:00,#NosUnimosONosJodemos\nLa revista Semana con s...,1528163832690003969,NaN,2022-05-22T00:00:00Z,25.0,2.0,58.0,0.0,None
3,es,1143717800,1528164148445585413,2022-05-22 00:01:39+00:00,@jarizabaletaf Federico Gutierrez Presidente d...,1528149422806384641,9.653315e+08,2022-05-22T00:00:00Z,17.0,1.0,35.0,0.0,None
4,es,61589689,1528164772281196544,2022-05-22 00:04:08+00:00,#NosUnimosONosJodemos porque no queremos en Co...,1528164772281196544,NaN,2022-05-22T00:00:00Z,20.0,1.0,21.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,qht,2406379355,1528525091578970112,2022-05-22 23:55:55+00:00,#NosUnimosONosJodemos #FedericoImparable #Fede...,1528525091578970112,NaN,2022-05-22T23:50:00Z,0.0,0.0,1.0,0.0,None
2433,qht,2406379355,1528523734658711552,2022-05-22 23:50:31+00:00,#NosUnimosONosJodemos #FedericoEsColombia #Fic...,1528523734658711552,NaN,2022-05-22T23:50:00Z,0.0,0.0,0.0,0.0,None
2434,es,899723221024591872,1528523946320138240,2022-05-22 23:51:22+00:00,@millersoto @FicoGutierrez @IvanDuque Solo son...,1528437012906364930,1.168275e+08,2022-05-22T23:50:00Z,0.0,0.0,0.0,0.0,None
2435,qht,575093546,1528525079998615553,2022-05-22 23:55:52+00:00,#NosUnimosONosJodemos https://t.co/8AezmjEar8,1528525079998615553,NaN,2022-05-22T23:50:00Z,0.0,0.0,0.0,0.0,None
